In [1]:
import os

In [2]:
%pwd

'/home/tinashe/codes/chicken2/coccidiosis_identification/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/home/tinashe/codes/chicken2/coccidiosis_identification'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    parameter_epochs: int
    parameter_batch_size: int
    parameter_is_augmentation: bool
    parameter_image_size: list


@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath:Path
    

In [6]:
from classifier.constants import *
from classifier.utils.common_functions import read_yaml, create_dirctories
import tensorflow as tf 

2024-12-10 11:14:03,937 - classifier - INFO - welcome tinashe


2024-12-10 11:14:08.107433: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath= CONFIG_FILE_PATH,
            parameters_filepath= PARAMETERS_FILE_PATH
            ):
       
        self.config= read_yaml(config_filepath)
        self.parameter= read_yaml(parameters_filepath)
        create_dirctories([self.config.artifacts_roots])


    def get_prepare_callbacks_config(self)-> PrepareCallbacksConfig:
        config=self.config.prepare_callbacks
        model_ckpt_dir= os.path.dirname(config.checkpoint_model_filepath)

        create_dirctories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callbacks_config= PrepareCallbacksConfig(
            root_dir= Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )
        return prepare_callbacks_config
    

    #initialising training configuration 
    def get_training_config(self)->TrainingConfig:
        training= self.config.training
        prepare_base_model= self.config.prepare_base_model
        parameter= self.parameter

        training_data=os.path.join(self.config.data_ingestion.unzip_dir,"chicken_images")
        create_dirctories([Path(training.root_dir)])

        training_config=TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path= Path(prepare_base_model.updated_base_model_path),
            training_data= Path(training_data),
            parameter_epochs= parameter.EPOCHS,
            parameter_batch_size= parameter.BATCH_SIZE,
            parameter_is_augmentation= parameter.AUGMENTATION,
            parameter_image_size=parameter.IMAGE_SIZE
       
        )
        return training_config

    



In [8]:
#creating componets
import time

In [9]:
class PrepareCallbacks:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config=config

    @property
    def _create_tb_callbacks(self):
        time_stamp= time.strftime("%Y-%M-%d-%H-%M-%S")
        tb_running_log_dir= os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{time_stamp}",

        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)

        


    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath= self.config.checkpoint_model_filepath,
            save_best_only= True
        )
    


    
    def get_tb_ckpt_callbacks(self):
        return[
            self._create_tb_callbacks,
            self._create_ckpt_callbacks,
        ]


In [10]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [11]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config= config


    def get_base_model(self):
        self.model=tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def train_valid_generator(self):
        datagenerator_kwargs= dict(
            rescale= 1./255,
            validation_split= 0.20
        )


        dataflow_kwargs= dict(
            target_size= self.config.parameter_image_size[:-1],
            batch_size=self.config.parameter_batch_size,
            interpolation= "bilinear"

        )

        valid_datagenerator=tf.keras.preprocessing.image.ImageDataGenerator(**datagenerator_kwargs)

        self.valid_generator= valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs

        )

        if self.config.parameter_is_augmentation:
            train_datagenerator= tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
    
            )

        else:
            train_datagenerator= valid_datagenerator

        self.train_generator= train_datagenerator.flow_from_directory(
            directory= self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    def train(self, callback_list: list):
        self.steps_per_epoch= self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps= self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.parameter_epochs,
            steps_per_epoch= self.steps_per_epoch,
            validation_steps= self.validation_steps,
            validation_data=self.valid_generator,
            callbacks= callback_list
        )

        self.save_model(
            path= self.config.trained_model_path,
            model=self.model
        )







In [12]:
#training pipeline 

try:
    config=ConfigurationManager()
    prepare_callbacks_config= config.get_prepare_callbacks_config()
    prepare_callbacks= PrepareCallbacks(config=prepare_callbacks_config)
    callback_list= prepare_callbacks.get_tb_ckpt_callbacks()

    training_config= config.get_training_config()
    training= Training(config= training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(
        callback_list=callback_list
    )

except Exception as e:
    raise e

2024-12-10 11:14:17,737 - classifier - INFO - yaml file: config/config.yaml loaded succefully
2024-12-10 11:14:17,764 - classifier - INFO - yaml file: parameters.yaml loaded succefully
2024-12-10 11:14:17,769 - classifier - INFO - created a directory at: artifacts
2024-12-10 11:14:17,770 - classifier - INFO - created a directory at: artifacts/prepare_callbacks/checkpoint_dir
2024-12-10 11:14:17,773 - classifier - INFO - created a directory at: artifacts/prepare_callbacks/tensorboard_log_dir
2024-12-10 11:14:17,779 - classifier - INFO - created a directory at: artifacts/training
Found 78 images belonging to 2 classes.
Found 312 images belonging to 2 classes.
Epoch 1/2


/home/tinashe/.conda/envs/chicken_tools/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
2024-12-10 11:14:26.658934: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 205520896 exceeds 10% of free system memory.
2024-12-10 11:14:27.694610: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 205520896 exceeds 10% of free system memory.
2024-12-10 11:14:32.014366: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 51380224 exceeds 10% of free system memory.
2024-12-10 11:14:32.127364: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 102760448 exceeds 10% of free system memory.
2024-12-

19/19 ━━━━━━━━━━━━━━━━━━━━ 695s 36s/step - accuracy: 0.5409 - loss: 9.5583 - val_accuracy: 0.3906 - val_loss: 7.2112
Epoch 2/2
 1/19 ━━━━━━━━━━━━━━━━━━━━ 8:54 30s/step - accuracy: 0.3750 - loss: 7.5413

2024-12-10 11:26:25.364091: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/home/tinashe/.conda/envs/chicken_tools/lib/python3.12/contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


19/19 ━━━━━━━━━━━━━━━━━━━━ 56s 1s/step - accuracy: 0.3750 - loss: 7.5413 - val_accuracy: 0.0000e+00 - val_loss: 52.8195


2024-12-10 11:26:51.537138: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
